<a href="https://colab.research.google.com/github/taosu-chatgpt/Colab-Free-PDF-Translator/blob/main/WSJ_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title 1. 初始化环境 & 注入防崩溃疫苗
# ==============================================================================
# 功能：安装依赖，修复 T4 显卡识别问题，并修改 BabelDOC 源码以防止超时崩溃。
# ==============================================================================
import os
import re
import time

print("🏗️ [1/3] 正在搭建 WSJ 翻译流水线...")

# 1. 系统级依赖安装 (让 Colab 看见显卡)
os.system("sudo apt-get install -y pciutils lshw zstd")

# 2. Python 依赖修复 (强制降级 Numpy 以兼容 PaddleOCR)
print("🔧 正在锁定依赖库版本...")
os.system("pip install paddlepaddle-gpu")
os.system("pip install 'numpy<2.0' --force-reinstall")

# 3. 安装 BabelDOC
if not os.path.exists('BabelDOC'):
    print("📦 克隆 BabelDOC 仓库...")
    os.system("git clone https://github.com/funstory-ai/BabelDOC.git")
    os.system("pip install -r BabelDOC/requirements.txt")
    os.system("pip install -e BabelDOC")

# 4. 安装 Ollama 服务
print("⬇️ 安装 Ollama AI 引擎...")
os.system("curl -fsSL https://ollama.com/install.sh | sh")

# 5. 【核心】注入“源码级防崩溃补丁”
# 说明：BabelDOC 原生代码没有超时重试机制，遇到难啃的段落会卡死。
# 我们直接用正则替换 translator.py 中的核心函数，植入“超时熔断+重试”逻辑。
print("💉 正在执行源码手术 (Timeout Patch)...")
target_file = '/content/BabelDOC/babeldoc/translator/translator.py'

if os.path.exists(target_file):
    with open(target_file, 'r') as f:
        code = f.read()

    # 定义“无敌版”翻译函数
    robust_func = """def do_llm_translate(self, text, system_prompt=None):
        import time
        max_retries = 3
        for attempt in range(max_retries):
            try:
                messages = []
                if system_prompt:
                    messages.append({"role": "system", "content": system_prompt})
                messages.append({"role": "user", "content": text})

                # [补丁] 强制 120秒 超时，防止死锁
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    temperature=0.1,  # 绝对理性模式
                    timeout=120       # 熔断时间
                )
                return response.choices[0].message.content
            except Exception as e:
                print(f"\\n🛡️ [护盾激活] 翻译尝试 {attempt+1}/{max_retries} 失败，自动重试...")
                time.sleep(2)

        print("❌ [跳过] 该段落过于复杂或含乱码，已跳过。")
        return "[原文翻译失败，已跳过]"
"""
    # 正则替换
    pattern = r"def do_llm_translate\(self, text, system_prompt=None\):[\s\S]*?return response\.choices\[0\]\.message\.content"
    new_code = re.sub(pattern, robust_func, code)

    with open(target_file, 'w') as f:
        f.write(new_code)
    print("✅ 补丁注入完成！系统已升级为“韧性模式”。")
else:
    print("❌ 警告：未找到源码文件，补丁可能未生效。")

print("\n🎉 环境准备就绪！请【重启会话 (Restart Session)】后运行第 2 步。")

🏗️ [1/3] 正在搭建 WSJ 翻译流水线...
🔧 正在锁定依赖库版本...
⬇️ 安装 Ollama AI 引擎...
💉 正在执行源码手术 (Timeout Patch)...
✅ 补丁注入完成！系统已升级为“韧性模式”。

🎉 环境准备就绪！请【重启会话 (Restart Session)】后运行第 2 步。


In [1]:
# @title 2. 启动服务 & 配置 Google 翻译模型
# ==============================================================================
# 功能：启动后台服务，并创建一个专为 WSJ 优化的 TranslateGemma-32k 模型。
# ==============================================================================
import subprocess
import time
import requests

print("🚀 [2/3] 正在预热 AI 引擎...")

# 1. 启动后台服务
subprocess.run(["pkill", "ollama"]) # 清理旧进程
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# 等待服务响应
print("⏳ 等待 Ollama 服务上线...", end="")
for i in range(20):
    try:
        if requests.get("http://localhost:11434").status_code == 200:
            print("\n✅ 服务已连通！")
            break
    except:
        time.sleep(2)
        print(".", end="")

# 2. 模型组装车间
print("💎 正在组装 Google TranslateGemma (4B)...")
subprocess.run(["ollama", "pull", "translategemma:4b"])

print("⚙️ 正在应用 WSJ 专用参数 (32k Context)...")
# 创建一个名为 "wsj-translator" 的定制模型
with open("Modelfile_wsj", "w") as f:
    f.write("FROM translategemma:4b\n")
    f.write("PARAMETER num_ctx 32768\n")      # 32k 超大显存窗口
    f.write("PARAMETER repeat_penalty 1.2\n") # 防止复读机死循环
    f.write("PARAMETER temperature 0.1")      # 降低创造性，提高准确度
subprocess.run(["ollama", "create", "wsj-translator", "-f", "Modelfile_wsj"])

print("\n🎉 专用翻译引擎 'wsj-translator' 准备就绪！")

🚀 [2/3] 正在预热 AI 引擎...
⏳ 等待 Ollama 服务上线....
✅ 服务已连通！
💎 正在组装 Google TranslateGemma (4B)...
⚙️ 正在应用 WSJ 专用参数 (32k Context)...

🎉 专用翻译引擎 'wsj-translator' 准备就绪！


In [3]:
# @title 3. (完美修正版) 自动接力翻译 + 自动重命名防覆盖
import os
import subprocess
import time
import shutil
from google.colab import files

# === 用户配置 ===
BATCH_SIZE = 5    # 每次翻译 5 页
TOTAL_PAGES = 20  # 总共翻译 20 页

# 1. 上传文件
print("📂 请上传 WSJ.pdf...")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("❌ 未上传文件")
filename = list(uploaded.keys())[0]
base_name = os.path.splitext(filename)[0] # 去掉 .pdf 后缀

# 工具函数：重启服务
def reset_engine():
    print("🧹 [内存回收] 正在重启服务以释放显存...")
    subprocess.run(["pkill", "ollama"])
    time.sleep(2)
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(5)

print(f"\n⚔️ 启动最终接力：目标 {TOTAL_PAGES} 页，结果将自动重命名保存。")

for start in range(1, TOTAL_PAGES + 1, BATCH_SIZE):
    end = min(start + BATCH_SIZE - 1, TOTAL_PAGES)
    page_range = f"{start}-{end}"

    # 1. 清洗引擎
    reset_engine()

    print(f"\n🚀 [第 {page_range} 页] 正在翻译中...")

    # 2. 构造命令
    cmd = f"""
    babeldoc \
      --files "{filename}" \
      --pages "{page_range}" \
      --openai \
      --openai-model "wsj-translator" \
      --openai-base-url "http://localhost:11434/v1" \
      --openai-api-key "ollama" \
      --lang-out zh \
      --no-auto-extract-glossary \
      --custom-system-prompt "Translate into Simplified Chinese."
    """

    # 3. 执行翻译
    exit_code = os.system(cmd)

    # 4. 【关键修正】自动重命名防止覆盖
    if exit_code == 0:
        # 原始输出文件名 (BabelDOC 默认生成的)
        original_dual = f"{base_name}.zh.dual.pdf"
        original_mono = f"{base_name}.zh.mono.pdf"

        # 我们想要的新文件名
        new_dual = f"WSJ_Part_{page_range}_Dual.pdf"
        new_mono = f"WSJ_Part_{page_range}_Mono.pdf"

        # 重命名并移动 (如果存在)
        if os.path.exists(original_dual):
            shutil.move(original_dual, new_dual)
            print(f"✅ 双语版已保存为: {new_dual}")

        if os.path.exists(original_mono):
            shutil.move(original_mono, new_mono)
            print(f"✅ 中文版已保存为: {new_mono}")
    else:
        print(f"⚠️ 第 {page_range} 页似乎遇到问题。")

print("\n🏁🏁🏁 全部完成！请看左侧文件列表，下载所有 'WSJ_Part_...' 开头的文件。")

📂 请上传 WSJ.pdf...


Saving WSJ.pdf to WSJ (2).pdf

⚔️ 启动最终接力：目标 20 页，结果将自动重命名保存。
🧹 [内存回收] 正在重启服务以释放显存...

🚀 [第 1-5 页] 正在翻译中...
✅ 双语版已保存为: WSJ_Part_1-5_Dual.pdf
✅ 中文版已保存为: WSJ_Part_1-5_Mono.pdf
🧹 [内存回收] 正在重启服务以释放显存...

🚀 [第 6-10 页] 正在翻译中...
✅ 双语版已保存为: WSJ_Part_6-10_Dual.pdf
✅ 中文版已保存为: WSJ_Part_6-10_Mono.pdf
🧹 [内存回收] 正在重启服务以释放显存...

🚀 [第 11-15 页] 正在翻译中...
✅ 双语版已保存为: WSJ_Part_11-15_Dual.pdf
✅ 中文版已保存为: WSJ_Part_11-15_Mono.pdf
🧹 [内存回收] 正在重启服务以释放显存...

🚀 [第 16-20 页] 正在翻译中...
✅ 双语版已保存为: WSJ_Part_16-20_Dual.pdf
✅ 中文版已保存为: WSJ_Part_16-20_Mono.pdf

🏁🏁🏁 全部完成！请看左侧文件列表，下载所有 'WSJ_Part_...' 开头的文件。
